In [1]:
pip install tweepy pandas

In [9]:
import tweepy
import pandas as pd

# Twitter API credentials
API_KEY = 'SP0XwJ5EROzvRU4I3Q6PCsv5V'
API_SECRET = 'gDy2QQcahM53IQBnGTsUCqbrj34byEClDZsLqKGrZyhpIDJQxs'
ACCESS_TOKEN = '1865077693152399360-2o9nHgmx8aGh3vm0mhhuBgDry6MpYy'
ACCESS_TOKEN_SECRET = '1Dix2OUzDmudONjyi6EZl61WL4x0ZeNySjjlJdlkYrkT0'

# Replace with your actual bearer token
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAI2ixQEAAAAAdWINS56uP7J5Lo5vhoEA5FZ%2B7Ic%3DzXbnMV1XQFbNwd2zgpCFAH4w74HpEUzXy7xMxprRutGIPOuhTc'

# Authenticate to Twitter
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=API_KEY,
    consumer_secret=API_SECRET,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET
)

# Define search query
query = "stock market"

# Fetch recent tweets
try:
    response = client.search_recent_tweets(
        query=query,
        max_results=100,
        tweet_fields=['created_at', 'text', 'author_id', 'public_metrics']
    )

    # Extract data if tweets are found
    if response.data:
        data = []
        for tweet in response.data:
            data.append({
                'created_at': tweet.created_at,
                'text': tweet.text,
                'user': tweet.author_id,
                'retweet_count': tweet.public_metrics['retweet_count'],
                'like_count': tweet.public_metrics['like_count']
            })

        # Save to a CSV file
        df = pd.DataFrame(data)
        df.to_csv("tweets.csv", index=False)
        print("Tweets saved to tweets.csv")
    else:
        print("No tweets found for the query.")

except Exception as e:
    print(f"An error occurred: {e}")


Tweets saved to tweets.csv


In [10]:
import nltk

# Download the 'punkt_tab' resource before using it
nltk.download('punkt_tab')

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load data
df = pd.read_csv("tweets.csv")

# Preprocessing function
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

df['cleaned_text'] = df['text'].apply(clean_text)
df.to_csv("cleaned_tweets.csv", index=False)
print("Cleaned tweets saved to cleaned_tweets.csv")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Cleaned tweets saved to cleaned_tweets.csv


In [11]:
pip install nltk


In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

# Analyze sentiment
df['sentiment'] = df['cleaned_text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
df['label'] = df['sentiment'].apply(lambda x: 'positive' if x > 0 else 'negative' if x < 0 else 'neutral')

df.to_csv("sentiment_tweets.csv", index=False)
print("Sentiment analysis saved to sentiment_tweets.csv")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Sentiment analysis saved to sentiment_tweets.csv


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Prepare data
X = df[['sentiment', 'retweet_count', 'like_count']].fillna(0)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         4
     neutral       1.00      1.00      1.00         9
    positive       1.00      1.00      1.00         7

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [14]:
pip install transformers


In [16]:
from transformers import pipeline

# Load FinBERT model for sentiment analysis
sentiment_pipeline = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")

# Analyze sentiment
# Replacing df_tweets with df as it seems to be the intended DataFrame
df['sentiment'] = df['cleaned_text'].apply(lambda x: sentiment_pipeline(x)[0]['label'])
df.to_csv("tweets_with_sentiment.csv", index=False)
print("Sentiment analysis completed and saved to tweets_with_sentiment.csv")

Sentiment analysis completed and saved to tweets_with_sentiment.csv


In [17]:
pip install plotly


In [28]:
import plotly.express as px
import pandas as pd

# Assuming 'df' contains your tweet data with a 'sentiment' and 'created_at' column

# 1. Group by 'created_at' and count sentiment occurrences
sentiment_summary = df.groupby(['created_at', 'sentiment'])['sentiment'].count().reset_index(name='count')


# 2. Ensure 'created_at' is in datetime format
sentiment_summary['created_at'] = pd.to_datetime(sentiment_summary['created_at'])

# 3. Create the Plotly Express line chart
fig = px.line(
    sentiment_summary,
    x="created_at",  # Specify x-axis as 'created_at'
    y="count",       # Specify y-axis as 'count'
    color="sentiment",# Specify color grouping by 'sentiment'
    title="Sentiment Trends Over Time",
    labels={'count': 'Count'}
)
fig.show()

In [34]:
import plotly.express as px
import pandas as pd

# Assuming 'df' contains your tweet data with 'created_at' and 'label' columns
df['created_at'] = pd.to_datetime(df['created_at'])  # Convert to datetime
sentiment_summary = df.groupby([pd.Grouper(key='created_at', freq='D'), 'label'])['label'].count().reset_index(name='count')

# Define custom color map
color_map = {'positive': 'pink',  # Baby pink
            'negative': '#C8A2C8',  # Lilac (using hex code)
            'neutral': '#FF69B4'}  # Rose pink (using hex code)

# Visualize sentiment trends over time using a bar graph
fig = px.bar(sentiment_summary, x='created_at', y='count', color='label',
             title="Sentiment Trends Over Time", labels={'count': 'Count'},
             barmode='group',
             color_discrete_map=color_map)  # Apply custom color map

# Increase space between bars and bar groups
fig.update_layout(bargap=0.4, bargroupgap=0.2)  # Adjust values as needed

fig.show()